# Imports

In [1]:
import numpy as np                     
import pandas as pd
import random as rand
from scipy.signal import correlate, correlation_lags
from scipy.ndimage import gaussian_filter1d

import matplotlib.pyplot as plt       
from matplotlib.patches import FancyArrow, Patch, Circle
import matplotlib.colors as mcolors
from matplotlib.colors import Normalize
import matplotlib.patches as patches
from matplotlib.lines import Line2D

import braingeneers                  
from braingeneers.analysis.analysis import SpikeData, read_phy_files, load_spike_data, burst_detection, randomize_raster
import braingeneers.data.datasets_electrophysiology as ephys
from multiprocessing import Pool
from tqdm import tqdm

# Data

In [2]:
sd = read_phy_files('/workspaces/human_hippocampus/data/ephys/2023-04-02-e-hc328_unperturbed/derived/kilosort2/hc3.28_hckcr1_chip16835_plated34.2_rec4.2_curated.zip')

/tmp/ipykernel_64595/653605344.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) read_phy_files. (Prefer load_spike_data()) -- Deprecated since version 0.1.13.
  sd = read_phy_files('/workspaces/human_hippocampus/data/ephys/2023-04-02-e-hc328_unperturbed/derived/kilosort2/hc3.28_hckcr1_chip16835_plated34.2_rec4.2_curated.zip')


# Code

In [3]:
def calculate_mean_firing_rates(spike_data):
    # Compute mean firing rates for each neuron
    firing_rates = [len(train) / spike_data.length for train in spike_data.train]
    return firing_rates

def get_neuron_positions(spike_data):
    # Extract neuron positions from spike_data
    neuron_x = []
    neuron_y = []
    for neuron in spike_data.neuron_data[0].values():
        neuron_x.append(neuron['position'][0])
        neuron_y.append(neuron['position'][1])
    neuron_positions = np.array([neuron_x, neuron_y]).T
    return neuron_positions

def precalculate_distances_angles(neuron_positions):
    # Vectorized calculation of distances
    diff = neuron_positions[:, np.newaxis, :] - neuron_positions[np.newaxis, :, :]
    distances = np.sqrt(np.sum(diff**2, axis=2))
    
    # Vectorized calculation of angles
    angles = np.arctan2(diff[..., 1], diff[..., 0]) % (2 * np.pi)
    
    return distances, angles


def create_reverse_rank_lookup(event_ranks):
    """
    Create a reverse lookup table for event ranks.
    
    Parameters:
    - event_ranks: A dictionary mapping (neuron_id, spike_time) to event rank.
    
    Returns:
    - A dictionary mapping event rank to (neuron_id, spike_time).
    """
    reverse_lookup = {rank: (neuron_id, spike_time) for (neuron_id, spike_time), rank in event_ranks.items()}
    return reverse_lookup 

def calculate_event_ranks(spike_data, precision=5):
    # Flatten the list of spikes, rounding spike times, and sort by time
    # Include the original index of each spike for uniqueness
    all_spikes = [(neuron_id, round(spike_time, precision), idx) 
                  for neuron_id, spikes in enumerate(spike_data.train) 
                  for idx, spike_time in enumerate(spikes)]
                  
    # Sort by neuron_id, then rounded spike time, then original index
    all_spikes_sorted = sorted(all_spikes, key=lambda x: (x[1], x[0], x[2]))
    
    # Generate a dictionary with event rank as key, (neuron_id, spike_time) as value
    ranks = {rank: (neuron_id, spike_time) for rank, (neuron_id, spike_time, _) in enumerate(all_spikes_sorted)}
    
    print(f"Total unique events: {len(ranks)}")
    
    return ranks

def precompute_close_neurons(distances, window_size=17.5):
    close_neurons = {}
    for i in range(len(distances)):
        close_neurons[i] = [j for j in range(len(distances)) if i != j and distances[i, j] < window_size]
    return close_neurons

In [4]:
def create_histograms_for_events(spike_data, event_ranks, spatial_range=(82, 1092), time_window_rank=30, bins=6):
    total_events = len(event_ranks)

    histograms_for_each_event = {}

    distances = precalculate_distances_angles(get_neuron_positions(spike_data))[0]
    angles = precalculate_distances_angles(get_neuron_positions(spike_data))[1]

    distance_bins = np.linspace(0, np.max(distances), bins+1)
    angle_bins = np.linspace(0, 2*np.pi, bins+1)

    print_every_n = max(total_events // 100, 1)  # Update progress every 10% or at least once
    
    for current_event_id in event_ranks:
        if current_event_id % print_every_n == 0:
            print(f"Processing event {current_event_id + 1}/{total_events}...")
            
        start_rank = max(0, current_event_id - time_window_rank)
        end_rank = min(total_events, current_event_id + time_window_rank + 1)

        event_distances, event_angles = [], []

        # Only consider events within the rank window
        for other_event_id in range(start_rank, end_rank):
            if other_event_id == current_event_id:
                continue

            current_neuron_id = event_ranks[current_event_id][0]
            other_neuron_id = event_ranks[other_event_id][0]

            # Lookup distance and angle between the two neurons
            distance = distances[current_neuron_id, other_neuron_id]
            angle = angles[current_neuron_id, other_neuron_id]

            if spatial_range[0] < distance < spatial_range[1]:
                event_distances.append(distance)
                event_angles.append(angle)

        distance_hist, _ = np.histogram(event_distances, bins=distance_bins)
        angle_hist, _ = np.histogram(event_angles, bins=angle_bins)

        histograms_for_each_event[current_event_id] = {
            'distance': distance_hist,
            'angle': angle_hist
        }
    
    print("Processing complete")
    return histograms_for_each_event

In [5]:
event_ranks = calculate_event_ranks(sd)
histograms = create_histograms_for_events(sd, event_ranks)

Total unique events: 113477
Processing event 1/113477...
Processing event 1135/113477...
Processing event 2269/113477...


/tmp/ipykernel_64595/2324574115.py:10: DeprecationWarning: Call to deprecated function (or staticmethod) neuron_data. (Use NeuronAttributes instead of neuron_data, with the function load_spike_data())
  for neuron in spike_data.neuron_data[0].values():


Processing event 3403/113477...
Processing event 4537/113477...
Processing event 5671/113477...
Processing event 6805/113477...
Processing event 7939/113477...
Processing event 9073/113477...
Processing event 10207/113477...
Processing event 11341/113477...
Processing event 12475/113477...
Processing event 13609/113477...
Processing event 14743/113477...
Processing event 15877/113477...
Processing event 17011/113477...
Processing event 18145/113477...
Processing event 19279/113477...
Processing event 20413/113477...
Processing event 21547/113477...
Processing event 22681/113477...
Processing event 23815/113477...
Processing event 24949/113477...
Processing event 26083/113477...
Processing event 27217/113477...
Processing event 28351/113477...
Processing event 29485/113477...
Processing event 30619/113477...
Processing event 31753/113477...
Processing event 32887/113477...
Processing event 34021/113477...
Processing event 35155/113477...
Processing event 36289/113477...
Processing event

In [25]:
# def apply_sliding_window_average_optimized(spike_data, event_histograms, bins=6):
#     smoothed_histograms = {}
#     event_ranks = calculate_event_ranks(spike_data)
#     total_events = len(event_histograms)
#     close_neurons = precompute_close_neurons(precalculate_distances_angles(get_neuron_positions(spike_data))[0])
#     print_every_n = max(total_events // 100, 1)  # Update progress every 10% or at least once
    
#     for event in event_histograms:
#         current_event_id = event
#         if current_event_id % print_every_n == 0:
#             print(f"Processing event {current_event_id + 1}/{total_events}...")

#         current_histograms = event_histograms[event]
#         current_neuron_id = event_ranks[current_event_id][0]

#         summed_histogram = {'distance': np.zeros(bins), 'angle': np.zeros(bins)}
#         summed_histogram_count = 0

#         # Add the current histogram to the sum
#         summed_histogram['distance'] += current_histograms['distance']
#         summed_histogram['angle'] += current_histograms['angle']
#         summed_histogram_count += 1

#         # Skip events from neurons without close neighbors
#         if not close_neurons.get(current_neuron_id):
#             continue

#         # Iterate only over events from close neurons
#         for other_event_id in event_histograms:
#             if other_event_id == current_event_id:
#                 continue
#             other_histograms = event_histograms[other_event_id]
#             other_neuron_id = event_ranks[other_event_id][0]

#             if other_neuron_id in close_neurons.get(current_neuron_id, []):
#                 summed_histogram['distance'] += other_histograms['distance']
#                 summed_histogram['angle'] += other_histograms['angle']
#                 summed_histogram_count += 1

#         if summed_histogram_count > 0:
#             smoothed_histograms[current_event_id] = {'distance': summed_histogram['distance'] / summed_histogram_count, 
#                                                      'angle': summed_histogram['angle'] / summed_histogram_count}
        
#     return smoothed_histograms


In [54]:
def apply_sliding_window_average_optimized(spike_data, event_histograms, window_width=17.5, bins=6):
    smoothed_histograms = {}
    event_ranks = calculate_event_ranks(spike_data)
    total_events = len(event_histograms)
    close_neurons = precompute_close_neurons(precalculate_distances_angles(get_neuron_positions(spike_data))[0], window_size=window_width / 2)
    print_every_n = max(total_events // 100, 1)  # Update progress every 10% or at least once
    
    for event_id, histograms in event_histograms.items():
        if event_id % print_every_n == 0:
            print(f"Processing event {event_id + 1}/{total_events}...")
        
        current_neuron_id = event_ranks[event_id][0]
        
        # Initialize summed histogram for the current event
        summed_histogram = {'distance': np.zeros(bins), 'angle': np.zeros(bins)}
        summed_histogram_count = 1  # Start count at 1 to include the current event
        
        # Add the histograms of the current event to the sum
        summed_histogram['distance'] += histograms['distance']
        summed_histogram['angle'] += histograms['angle']

        # Check for close neighbors and add their histograms to the sum
        for neighbor_id in close_neurons.get(current_neuron_id, []):
            if neighbor_id != event_id and neighbor_id in event_histograms:
                neighbor_histograms = event_histograms[neighbor_id]
                summed_histogram['distance'] += neighbor_histograms['distance']
                summed_histogram['angle'] += neighbor_histograms['angle']
                summed_histogram_count += 1

        # Compute the average histogram
        if summed_histogram_count > 0:
            smoothed_histograms[event_id] = {
                'distance': summed_histogram['distance'] / summed_histogram_count,
                'angle': summed_histogram['angle'] / summed_histogram_count
            }
    
    return smoothed_histograms

In [51]:
1024 in histograms

True

In [26]:
histograms = create_histograms_for_events(sd, event_ranks)

/tmp/ipykernel_64595/2324574115.py:10: DeprecationWarning: Call to deprecated function (or staticmethod) neuron_data. (Use NeuronAttributes instead of neuron_data, with the function load_spike_data())
  for neuron in spike_data.neuron_data[0].values():


Processing event 1/113477...
Processing event 1135/113477...
Processing event 2269/113477...
Processing event 3403/113477...
Processing event 4537/113477...
Processing event 5671/113477...
Processing event 6805/113477...
Processing event 7939/113477...
Processing event 9073/113477...
Processing event 10207/113477...
Processing event 11341/113477...
Processing event 12475/113477...
Processing event 13609/113477...
Processing event 14743/113477...
Processing event 15877/113477...
Processing event 17011/113477...
Processing event 18145/113477...
Processing event 19279/113477...
Processing event 20413/113477...
Processing event 21547/113477...
Processing event 22681/113477...
Processing event 23815/113477...
Processing event 24949/113477...
Processing event 26083/113477...
Processing event 27217/113477...
Processing event 28351/113477...
Processing event 29485/113477...
Processing event 30619/113477...
Processing event 31753/113477...
Processing event 32887/113477...
Processing event 34021

In [55]:
averaged_histograms = apply_sliding_window_average_optimized(sd, histograms)

Total unique events: 113477
Processing event 1/113477...
Processing event 1135/113477...
Processing event 2269/113477...
Processing event 3403/113477...
Processing event 4537/113477...
Processing event 5671/113477...
Processing event 6805/113477...
Processing event 7939/113477...
Processing event 9073/113477...
Processing event 10207/113477...
Processing event 11341/113477...
Processing event 12475/113477...
Processing event 13609/113477...
Processing event 14743/113477...
Processing event 15877/113477...
Processing event 17011/113477...
Processing event 18145/113477...
Processing event 19279/113477...
Processing event 20413/113477...
Processing event 21547/113477...
Processing event 22681/113477...
Processing event 23815/113477...
Processing event 24949/113477...
Processing event 26083/113477...
Processing event 27217/113477...
Processing event 28351/113477...
Processing event 29485/113477...
Processing event 30619/113477...
Processing event 31753/113477...


/tmp/ipykernel_64595/2324574115.py:10: DeprecationWarning: Call to deprecated function (or staticmethod) neuron_data. (Use NeuronAttributes instead of neuron_data, with the function load_spike_data())
  for neuron in spike_data.neuron_data[0].values():


Processing event 32887/113477...
Processing event 34021/113477...
Processing event 35155/113477...
Processing event 36289/113477...
Processing event 37423/113477...
Processing event 38557/113477...
Processing event 39691/113477...
Processing event 40825/113477...
Processing event 41959/113477...
Processing event 43093/113477...
Processing event 44227/113477...
Processing event 45361/113477...
Processing event 46495/113477...
Processing event 47629/113477...
Processing event 48763/113477...
Processing event 49897/113477...
Processing event 51031/113477...
Processing event 52165/113477...
Processing event 53299/113477...
Processing event 54433/113477...
Processing event 55567/113477...
Processing event 56701/113477...
Processing event 57835/113477...
Processing event 58969/113477...
Processing event 60103/113477...
Processing event 61237/113477...
Processing event 62371/113477...
Processing event 63505/113477...
Processing event 64639/113477...
Processing event 65773/113477...
Processing

In [61]:
large_averaged_histograms = apply_sliding_window_average_optimized(sd, histograms, window_width=336b)

Total unique events: 113477
Processing event 1/113477...
Processing event 1135/113477...
Processing event 2269/113477...
Processing event 3403/113477...
Processing event 4537/113477...
Processing event 5671/113477...
Processing event 6805/113477...


/tmp/ipykernel_64595/2324574115.py:10: DeprecationWarning: Call to deprecated function (or staticmethod) neuron_data. (Use NeuronAttributes instead of neuron_data, with the function load_spike_data())
  for neuron in spike_data.neuron_data[0].values():


Processing event 7939/113477...
Processing event 9073/113477...
Processing event 10207/113477...
Processing event 11341/113477...
Processing event 12475/113477...
Processing event 13609/113477...
Processing event 14743/113477...
Processing event 15877/113477...
Processing event 17011/113477...
Processing event 18145/113477...
Processing event 19279/113477...
Processing event 20413/113477...
Processing event 21547/113477...
Processing event 22681/113477...
Processing event 23815/113477...
Processing event 24949/113477...
Processing event 26083/113477...
Processing event 27217/113477...
Processing event 28351/113477...
Processing event 29485/113477...
Processing event 30619/113477...
Processing event 31753/113477...
Processing event 32887/113477...
Processing event 34021/113477...
Processing event 35155/113477...
Processing event 36289/113477...
Processing event 37423/113477...
Processing event 38557/113477...
Processing event 39691/113477...
Processing event 40825/113477...
Processing e

In [65]:
def subtract_average_histograms(spike_data, averaged_histograms, bins=6):
    final_histograms = {}
    window_histograms = apply_sliding_window_average_optimized(spike_data, averaged_histograms, window_width=(336/2), bins=6)
    
    final_histograms = {event_id: {
        'distance': histograms['distance'] - 0.1*window_histograms[event_id]['distance'],
        'angle': histograms['angle'] - 0.1*window_histograms[event_id]['angle']
    } for event_id, histograms in averaged_histograms.items()}

    return final_histograms

In [66]:
final_histograms = subtract_average_histograms(sd, averaged_histograms)

Total unique events: 113477
Processing event 1/113477...
Processing event 1135/113477...
Processing event 2269/113477...
Processing event 3403/113477...
Processing event 4537/113477...
Processing event 5671/113477...
Processing event 6805/113477...
Processing event 7939/113477...
Processing event 9073/113477...
Processing event 10207/113477...
Processing event 11341/113477...
Processing event 12475/113477...
Processing event 13609/113477...
Processing event 14743/113477...
Processing event 15877/113477...


/tmp/ipykernel_64595/2324574115.py:10: DeprecationWarning: Call to deprecated function (or staticmethod) neuron_data. (Use NeuronAttributes instead of neuron_data, with the function load_spike_data())
  for neuron in spike_data.neuron_data[0].values():


Processing event 17011/113477...
Processing event 18145/113477...
Processing event 19279/113477...
Processing event 20413/113477...
Processing event 21547/113477...
Processing event 22681/113477...
Processing event 23815/113477...
Processing event 24949/113477...
Processing event 26083/113477...
Processing event 27217/113477...
Processing event 28351/113477...
Processing event 29485/113477...
Processing event 30619/113477...
Processing event 31753/113477...
Processing event 32887/113477...
Processing event 34021/113477...
Processing event 35155/113477...
Processing event 36289/113477...
Processing event 37423/113477...
Processing event 38557/113477...
Processing event 39691/113477...
Processing event 40825/113477...
Processing event 41959/113477...
Processing event 43093/113477...
Processing event 44227/113477...
Processing event 45361/113477...
Processing event 46495/113477...
Processing event 47629/113477...
Processing event 48763/113477...
Processing event 49897/113477...
Processing

In [67]:
final_histograms

{0: {'distance': array([ 6.73333333,  9.9       ,  5.73333333, -0.1       ,  0.        ,
          0.        ]),
  'angle': array([ 8.66666667,  9.9       , -0.73333333, -0.5       , -0.7       ,
          5.63333333])},
 1: {'distance': array([ 3.85 , 13.2  ,  7.725, -0.1  ,  0.   ,  0.   ]),
  'angle': array([ 4.75 ,  1.175, -0.425,  0.8  , 13.825,  4.55 ])},
 2: {'distance': array([10.73333333,  8.43333333,  4.36666667,  0.83333333,  0.        ,
          0.        ]),
  'angle': array([ 1.4       , 11.63333333,  5.76666667,  3.63333333,  0.3       ,
          1.63333333])},
 3: {'distance': array([ 8.1875,  8.6625,  7.1   , -0.05  ,  0.    ,  0.    ]),
  'angle': array([ 5.5375, 17.3375,  0.85  , -0.15  , -2.075 ,  2.4   ])},
 4: {'distance': array([13.12, 10.22,  8.53,  0.82,  0.  ,  0.  ]),
  'angle': array([ 2.72,  6.94, -0.36,  2.75, 19.88,  0.76])},
 5: {'distance': array([14.54, 11.06,  4.14, -0.04,  0.  ,  0.  ]),
  'angle': array([ 2.72, 14.9 ,  1.24,  1.42,  2.92,  6.5 ])}

In [ ]:
# def subtract_average_histograms(spike_data, averaged_histograms, bins=6):
#     final_histograms = {}
#     window_histograms = {}
#     event_ranks = calculate_event_ranks(spike_data)
#     total_events = len(averaged_histograms)
#     close_neurons = precompute_close_neurons(precalculate_distances_angles(get_neuron_positions(spike_data))[0], window_size=336)

#     for event in averaged_histograms:
#         current_event_id = event
#         current_histograms = averaged_histograms[event]
#         current_neuron_id = event_ranks[current_event_id][0]

#         window_histograms[current_event_id] = {'distance': np.zeros(bins), 'angle': np.zeros(bins)}
#         window_histograms_count = 0

#         # Skip events from neurons without close neighbors
#         if not close_neurons.get(current_neuron_id):
#             continue

#         # Iterate only over events from close neurons
#         for other_event_id in averaged_histograms:
#             if other_event_id == current_event_id:
#                 continue
#             other_histograms = averaged_histograms[other_event_id]
#             other_neuron_id = event_ranks[other_event_id][0]

#             if other_neuron_id in close_neurons.get(current_neuron_id, []):
#                 window_histograms[current_event_id]['distance'] += other_histograms['distance']
#                 window_histograms[current_event_id]['angle'] += other_histograms['angle']
#                 window_histograms_count += 1
            
#         if window_histograms_count > 0:
#             window_histograms[current_event_id]['distance'] /= window_histograms_count
#             window_histograms[current_event_id]['angle'] /= window_histograms_count

#     final_histograms = averaged_histograms - window_histograms
#     return final_histograms